### 単純な多数決分類器を実装する

In [1]:
import numpy as np
np.argmax(np.bincount([0,0,1], weights=[0.2, 0.2, 0.6]))

1

In [3]:
ex = np.array([[0.9, 0.1],
                         [0.8, 0.2],
                         [0.4, 0.6]])

In [4]:
p = np.average(ex, axis=0, weights=[0.2, 0.2, 0.6])
p

array([0.58, 0.42])

In [5]:
np.argmax(p)

0